# Day 9: Sensor Boost
https://adventofcode.com/2019/day/9

## Part 1

In [ ]:
import numpy as np
import urllib.request
import math
import itertools

In [ ]:
class IntcodeParameter():
    POSITION = 0
    IMMEDIATE = 1
    RELATIVE = 2
    
    value = None
    mode = None
    
    def __init__(self, value, mode):
        self.value = value
        self.mode = mode
        assert mode in [IntcodeParameter.POSITION, IntcodeParameter.IMMEDIATE, IntcodeParameter.RELATIVE], f'UNKNOWN PARAMETER MODE {mode}'

    def checkBoundedParameter(self, program):
        if self.mode == IntcodeParameter.POSITION:
            assert self.value >= 0 and self.value < len(program.code), f'position parameter out of bounds. Position: {self.value}, Program size {len(program.code)}'
        elif self.mode == IntcodeParameter.RELATIVE:
            realPos = self.value + program.relative_base
            assert realPos >= 0 and realPos < len(program.code), f'relative parameter out of bounds. Position: {self.value}, Relative base {program.relative_base}, Program size {len(program.code)}'
        return True
        
    def evaluateParameter(self, program):
        if self.mode == IntcodeParameter.POSITION:
            return program.code[self.value]
        elif self.mode == IntcodeParameter.IMMEDIATE:
            return self.value
        elif self.mode == IntcodeParameter.RELATIVE:
            return program.code[self.value + program.relative_base]
        else:
            raise Exception('UNKNOWN PARAMETER MODE {}. Value {}. Program {}'.format(self.mode, self.value, program))
    
    def getParameterPosition(self, program):
        if self.mode == IntcodeParameter.POSITION:
            return self.value
        elif self.mode == IntcodeParameter.RELATIVE:
            return self.value + program.relative_base
        elif self.mode == IntcodeParameter.IMMEDIATE:
            raise Exception(f'Immediate parameter mode {self.value} cannot be used as position!')
        else:
            raise Exception(f'UNKNOWN PARAMETER MODE {self.mode}. Value {self.value}. Program {program}')
    
    def toString(self, program):
        return '[value {}, mode {}, evaluation {}]'.format(self.value, self.mode, self.evaluateParameter(program))

In [ ]:
class IntcodeInstruction():
    endComputations = False
    parameters = []
    
    def mustEndComputations(self):
        return self.endComputations
    
    def getInstructionSize(self):
        return len(self.parameters) + 1 #parameters + opcode
    
    def getNextInstructionPointer(self, program):
        newIP = program.IP + self.getInstructionSize()
        assert newIP >= 0 and newIP < len(program.code), f'new instruction pointer out of bounds. New IP {newIP}, last IP {program.IP}, Program size {len(program.code)}'
        return newIP
    
    def extractParameters(self, program):
        pass
    
    def performComputation(self, program):
        pass
    
    def checkParameters(self, program):
        for param in self.parameters:
            param.checkBoundedParameter(program)
    
    def loadParameters(self, program):
        self.parameters = self.extractParameters(program)
        self.checkParameters(program)
    
    def compute(self, program):
        self.performComputation(program)
    
    def toString(self, program):
        msg = f'{self.__class__.__name__} '
        for param in self.parameters:
            msg += param.toString(program)
        
        return (msg)

class OneParameterInst(IntcodeInstruction):
    def extractParameters(self, program):
        parameters = []
        modes = program.code[program.IP] // 100
        parameters.append(IntcodeParameter(program.code[program.IP + 1], modes % 10))
        
        return parameters

class TwoParametersInst(OneParameterInst):
    def extractParameters(self, program):
        parameters = super().extractParameters(program)
        modes = program.code[program.IP] // 100
        #1 parameters + 1 result
        modes //= 10
        parameters.append(IntcodeParameter(program.code[program.IP + 2], modes % 10))
        
        return parameters
    
class ThreeParameterInst(TwoParametersInst):
    def extractParameters(self, program):
        parameters = super().extractParameters(program)
        modes = program.code[program.IP] // 100
        #2 parameters + 1 result
        modes //= 10
        modes //= 10
        parameters.append(IntcodeParameter(program.code[program.IP + 3], modes % 10))
        
        return parameters
    
class AdditionInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = self.parameters[0].evaluateParameter(program) + self.parameters[1].evaluateParameter(program)
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue
        
class MultiplyInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = self.parameters[0].evaluateParameter(program) * self.parameters[1].evaluateParameter(program)
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue

class LessThanInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = 0
        if self.parameters[0].evaluateParameter(program) < self.parameters[1].evaluateParameter(program):
            resultValue = 1
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue
        
class EqualsInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = 0
        if self.parameters[0].evaluateParameter(program) == self.parameters[1].evaluateParameter(program):
            resultValue = 1
        program.code[self.parameters[2].getParameterPosition(program)] = resultValue
        
        
class InputInst(OneParameterInst):
    def performComputation(self, program):
        n = input('Solicitando dato de entrada:')
        
        # TODO: Validar dato!!! Debe ser entero, dado que luego podrá ser
        # usado para operar con él o ser un opcode, etc..
        
        program.code[self.parameters[0].getParameterPosition(program)] = int(n)

class OutputInst(OneParameterInst):
    def performComputation(self, program):
        print('>>>OUTPUT: {}'.format(self.parameters[0].evaluateParameter(program)))

class JumpIfTrueInst(TwoParametersInst):
    def performComputation(self, program):
#         print('JumpIfTrueInst!')
        pass

    def getNextInstructionPointer(self, program):
        if self.parameters[0].evaluateParameter(program) != 0:
            return self.parameters[1].evaluateParameter(program)
        
        return super().getNextInstructionPointer(program)

class JumpIfFalseInst(TwoParametersInst):
    def performComputation(self, program):
#         print('JumpIfFalseInst!')
        pass

    def getNextInstructionPointer(self, program):
        if self.parameters[0].evaluateParameter(program) == 0:
            return self.parameters[1].evaluateParameter(program)
        
        return super().getNextInstructionPointer(program)

class AdjustRelativeBaseInst(OneParameterInst):
    def performComputation(self, program):
        shift = self.parameters[0].evaluateParameter(program)
        program.relative_base += shift
    
class QuitInst(IntcodeInstruction):
    def __init__(self):
        self.endComputations = True

    def extractParameters(self, program):
        return []
        
    def performComputation(self, program):
        pass

In [ ]:
class IntcodeProgram():
    MAX_SIZE_IMAGE = 2000
    IP = None
    code = None
    relative_base = None
    
    def __init__(self, code, relative_base = 0):
        self.IP = 0
        self.relative_base = relative_base
        self.code = self.__expand_code(code) #maybe code.copy() ?

    def __expand_code(self, code):
        assert len(code) < IntcodeProgram.MAX_SIZE_IMAGE, f'Program is too big! program size {len(code)}. Max size {IntcodeProgram.MAX_SIZE_IMAGE}'
        return code + ( (IntcodeProgram.MAX_SIZE_IMAGE - len(code)) * [0]  )


In [ ]:
class IntcodeInterpreterV4():
    instructions = {}
    program = None
    
    def __init__(self):
        # TODO: Inject instruction set
        # Arithmetic
        self.instructions[1]  = AdditionInst
        self.instructions[2]  = MultiplyInst
        
        # IO
        self.instructions[3]  = InputInst
        self.instructions[4]  = OutputInst
        
        #Logic
        self.instructions[7]  = LessThanInst
        self.instructions[8]  = EqualsInst
        
        #Jump
        self.instructions[5]  = JumpIfTrueInst
        self.instructions[6]  = JumpIfFalseInst

        #Mem
        self.instructions[9]  = AdjustRelativeBaseInst
        
        #STOP
        self.instructions[99] = QuitInst

    def getInstruction(self, program):
        opcode = program.code[program.IP]
        opcode = opcode % 100
        if opcode in self.instructions:
            instruction = self.instructions[opcode]()
            instruction.loadParameters(program)
            return instruction
        else:
            raise Exception('UNKNOWN opcode {} ({}) at position {}.\nPROGRAM: {}'.format(opcode, program.code[program.IP], program.IP, program.code))
        return None
        
    def compute(self, ext_program):
#         print(f'Executing {ext_program}!')
        self.program = IntcodeProgram(ext_program)
        while True:
            # Extract next instruction            
            instruction = self.getInstruction(self.program)
                
            # Exit if it must
            if instruction.mustEndComputations():
                break
            else:
                # Otherwise, compute instruction
                instruction.compute(self.program)
                # Change IP (Instruction Pointer)
                self.program.IP = instruction.getNextInstructionPointer(self.program)
            
#             print('DEBUGGGGGG')
#             print(program)
#             print('IP {}'.format(self.IP))
#             print('DEBUGGGGGG')
            
        return ext_program

### Tests

#### Test 1

109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99 takes no input and produces a copy of itself as output.

In [ ]:
program = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]

IntcodeInterpreterV4().compute(program.copy())

#### Test 2
1102,34915192,34915192,7,4,7,99,0 should output a 16-digit number.

In [ ]:
program = [1102,34915192,34915192,7,4,7,99,0]

IntcodeInterpreterV4().compute(program.copy())

#### Test 3
104,1125899906842624,99 should output the large number in the middle.

In [ ]:
program = [104,1125899906842624,99]

IntcodeInterpreterV4().compute(program.copy())

### Solution

In [ ]:
input_9 = r'aoc2019-input-day9.txt'
with open(input_9, 'r') as f:
    data9 = [int(data) for data in f.read().split(',') if len(data) > 0]

In [ ]:
IntcodeInterpreterV4().compute(data9.copy())

Answer '1'

>>>OUTPUT: 2399197539

## Part 2

In [ ]:
IntcodeInterpreterV4().compute(data9.copy())

Answer '2'

>>>OUTPUT: 35106